<a href="https://colab.research.google.com/github/akanison12/Sudo-code-2024/blob/main/240107_Text_Preprocessing_with_NLTK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount Driver

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Download dataset from Kaggle

In [ ]:
!pip install kaggle
!kaggle datasets download -d haitranquangofficial/vietnamese-online-news-dataset
!pip install underthesea


Dataset URL: https://www.kaggle.com/datasets/haitranquangofficial/vietnamese-online-news-dataset
License(s): CC0-1.0
vietnamese-online-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


#Import Library

In [ ]:
import json
import re
import os
import zipfile
import csv
from underthesea import  sent_tokenize,text_normalize,word_tokenize


#Extract the zip file

In [ ]:


# Path to your zip file
zip_path = '/content/vietnamese-online-news-dataset.zip'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/extracted_dataset')

#Make File
extracted_path = '/content/extracted_dataset'
print(os.listdir(extracted_path))


['news_dataset.json']


#Preprocessing Text


In [ ]:

class TextPreprocessor:
    def __init__(self, stopword_file_path):
        """
        Khởi tạo TextPreprocessor với danh sách stopwords từ file.
        """
        self.stopwords_vietnamese = self.load_stopwords(stopword_file_path)

    def load_stopwords(self, file_path):
        """
        Đọc stopwords từ file.
        """
        with open(file_path, 'r', encoding='utf-8') as f:
            stopwords = set(line.strip() for line in f)
        return stopwords

    def preprocess_text(self, text):
        """
        Thực hiện tiền xử lý văn bản bao gồm:
        - Chuyển văn bản về chữ thường.
        - Loại bỏ ký tự đặc biệt và số.
        - Loại bỏ stopwords.
        - Lemmatization (đưa từ về dạng chuẩn).
        """

        text = text.lower()


        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[^\w\s]', '', text)


        words = word_tokenize(text)
        filtered_words = [word for word in words if word not in self.stopwords_vietnamese]


        lemmatized_text = ' '.join(filtered_words)

        return lemmatized_text




Apply Preprocessing into Dataset


In [ ]:
class ArticleProcessor:
    def __init__(self, json_file_path, stopword_file_path):
        """
        Khởi tạo ArticleProcessor với file JSON và file stopwords.
        """
        self.preprocessor = TextPreprocessor(stopword_file_path)
        self.articles = self.load_articles(json_file_path)

    def load_articles(self, json_file_path):
        """
        Đọc dữ liệu từ file JSON.
        """
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
        return data

    def process_articles(self):
        """

        Processeing mỗi bài báo, in ra từng nội dung và tiêu đề ban đầu và sau khi xử lý.
         Lưu kết quả vào file CSV.
        """

        #Tao ra file csv để ghi
        with open('processed_articles.csv', 'w', newline='') as csvfile:
            fieldnames = ['Document', 'Preprocessed Title', 'Preprocessed Content']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()

            for i, article in enumerate(self.articles):
                content = article['content']
                title = article['title']

                # Preprocess content and title
                preprocessed_content = self.preprocessor.preprocess_text(content)
                preprocessed_title = self.preprocessor.preprocess_text(title)

                # Ghi data vào csv
                writer.writerow({
                    'Document': i + 1,

                    'Preprocessed Title': preprocessed_title,
                    'Preprocessed Content': preprocessed_content
                })

                print(f"Document {i+1}:")
                print(f"Original Content:\n{content}\n")
                print(f"Original Title:\n{title}\n")
                print(f"Preprocessed Content:\n{preprocessed_content}\n")
                print(f"Preprocessed Title:\n{preprocessed_title}\n")
                print("\n---\n")
                #Dừng khi hết doucument
                if i == 33:
                    break

In [ ]:
# Đường dẫn tới file JSON và file stopwords
json_file_path = '/content/extracted_dataset/news_dataset.json'
stopword_file_path = '/content/drive/MyDrive/sudo-code-2024/W1/vietnamese-stopwords.txt'

# Tạo đối tượng ArticleProcessor và xử lý các bài viết
processor = ArticleProcessor(json_file_path, stopword_file_path)


#In ra kết quả đã processing
data_clean = processor.process_articles()








Document 1:
Original Content:
Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường, nhiều người